In [2]:
# -------------- #

import pandas as pd
import os
import datetime
import math
import load_data

from config import (
    RAW_DATA_DIR,
    LAG_DICT,
    SLIDING_DICT
)

from tqdm import tqdm
import numpy as np 

import seaborn as sns
import matplotlib.pyplot as plt
color = sns.color_palette()

import subprocess
from subprocess import check_output
import gc

%matplotlib inline

In [18]:
holidays_events_df = pd.read_csv(RAW_DATA_DIR+'holidays_events.csv')
items_df = pd.read_csv(RAW_DATA_DIR+'items.csv')
oil_df = pd.read_csv(RAW_DATA_DIR+'oil.csv',parse_dates=['date'])
stores_df = pd.read_csv(RAW_DATA_DIR+'stores.csv')
transactions_df = pd.read_csv(RAW_DATA_DIR+'transactions.csv')

df = pd.read_csv(
    'datasets/train.csv', usecols=[1, 2, 3, 4, 5],
    parse_dates=["date"]
)

test_df = pd.read_csv(
    'datasets/test.csv', usecols=[1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]
)

## Memory Deducion

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 5 columns):
date           datetime64[ns]
store_nbr      int64
item_nbr       int64
unit_sales     float64
onpromotion    object
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 4.7+ GB


In [19]:
#--- for integer type columns ---
def change_datatype(df):
    float_cols = list(df.select_dtypes(include=['int']).columns)
    for col in float_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
    return df
#--- for float type columns ---
def change_datatype_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)            
    return df

In [20]:
df = change_datatype(df)
df = change_datatype_float(df)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 5 columns):
date           datetime64[ns]
store_nbr      int8
item_nbr       int32
unit_sales     float32
onpromotion    object
dtypes: datetime64[ns](1), float32(1), int32(1), int8(1), object(1)
memory usage: 2.9+ GB


#### Reduce from 4.7GB to 2.9GB

## train.csv

In [24]:
df.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2013-01-01,25,103665,7.0,NaN
1,2013-01-01,25,105574,1.0,NaN
2,2013-01-01,25,105575,2.0,NaN
3,2013-01-01,25,108079,1.0,NaN
4,2013-01-01,25,108701,1.0,NaN


In [ ]:
print('The date range is from {} to {}.'.format())

## test.csv

### IMPORTANT

In test.csv, it tries to ask us to predict new items. 

So we train 2 separate models: one for old items. One for new item.



## store.csv

In [21]:
print('In the datasets, we have {} stores in total'.format(stores_df.shape[0]))
print('They are from {} different cities and {} states. Also, they can be classified into {} types and {} clusters'.format(
    stores_df.city.unique().shape[0],
    stores_df.state.unique().shape[0],
    stores_df.type.unique().shape[0],
    stores_df.cluster.unique().shape[0]
))

In the datasets, we have 54 stores in total


## items.csv

## oil.csv

## How many stores. How many items in each stores. Their growth

## 